<a href="https://colab.research.google.com/github/liuezhong/homework/blob/master/ca'p'stoneProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install lightgbm xgboost catboost category-encoders sklearn pandas==1.1.5

     |████████████████████████████████| 67.4 MB 25 kB/s 
     |████████████████████████████████| 80 kB 8.8 MB/s 


In [2]:
!git clone https://github.com/liuezhong/homework.git

Cloning into 'homework'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 22 (delta 4), reused 7 (delta 1), pack-reused 0
Unpacking objects: 100% (22/22), done.


In [3]:
import pandas as pd
import numpy as np

train = pd.read_csv('./homework/two/dataset/train_final.csv', engine='python')
test = pd.read_csv('./homework/two/dataset/test_final.csv', engine='python')


In [4]:
train.columns.to_list()

['continuous_annual_inc',
 'continuous_annual_inc_joint',
 'continuous_delinq_2yrs',
 'continuous_dti',
 'continuous_dti_joint',
 'continuous_fico_range_high',
 'continuous_fico_range_low',
 'continuous_funded_amnt',
 'continuous_funded_amnt_inv',
 'continuous_inq_last_6mths',
 'continuous_installment',
 'continuous_int_rate',
 'continuous_last_fico_range_high',
 'continuous_last_fico_range_low',
 'continuous_loan_amnt',
 'loan_status',
 'continuous_mths_since_last_delinq',
 'continuous_mths_since_last_major_derog',
 'continuous_mths_since_last_record',
 'continuous_open_acc',
 'continuous_pub_rec',
 'discrete_addr_state_1_one_hot',
 'discrete_addr_state_2_one_hot',
 'discrete_addr_state_3_one_hot',
 'discrete_addr_state_4_one_hot',
 'discrete_addr_state_5_one_hot',
 'discrete_addr_state_6_one_hot',
 'discrete_addr_state_7_one_hot',
 'discrete_addr_state_8_one_hot',
 'discrete_addr_state_9_one_hot',
 'discrete_addr_state_10_one_hot',
 'discrete_addr_state_11_one_hot',
 'discrete_addr_s

In [5]:
import lightgbm as lgb
train_dataset = lgb.Dataset(train.drop(columns='loan_status'), train['loan_status'])
test_dataset = lgb.Dataset(test.drop(columns='loan_status'), test['loan_status'])

In [6]:
tran_shaffle = train.sample(frac=1)
train_new = lgb.Dataset(train.drop(columns='loan_status'), train['loan_status'])

In [7]:
param = {'num_leaves': 31, 'objective': 'binary', 'metric':'binary_error'}
num_round = 2000

In [8]:
model = lgb.train(param, train_new, num_boost_round=num_round, valid_sets=[train_dataset, test_dataset])

[1]	valid_0's binary_error: 0.20424	valid_1's binary_error: 0.19548
[2]	valid_0's binary_error: 0.20424	valid_1's binary_error: 0.19548
[3]	valid_0's binary_error: 0.20424	valid_1's binary_error: 0.19548
[4]	valid_0's binary_error: 0.20424	valid_1's binary_error: 0.19548
[5]	valid_0's binary_error: 0.14838	valid_1's binary_error: 0.14168
[6]	valid_0's binary_error: 0.112	valid_1's binary_error: 0.11048
[7]	valid_0's binary_error: 0.099	valid_1's binary_error: 0.10016
[8]	valid_0's binary_error: 0.09384	valid_1's binary_error: 0.09572
[9]	valid_0's binary_error: 0.08812	valid_1's binary_error: 0.09036
[10]	valid_0's binary_error: 0.08416	valid_1's binary_error: 0.08636
[11]	valid_0's binary_error: 0.08172	valid_1's binary_error: 0.08476
[12]	valid_0's binary_error: 0.08048	valid_1's binary_error: 0.08422
[13]	valid_0's binary_error: 0.07882	valid_1's binary_error: 0.08354
[14]	valid_0's binary_error: 0.07806	valid_1's binary_error: 0.0832
[15]	valid_0's binary_error: 0.07744	valid_1's b

In [9]:
import io
import multiprocessing
from contextlib import redirect_stdout
from copy import deepcopy
from dataclasses import dataclass, asdict
import hyperopt.pyll
from hyperopt import fmin, tpe, hp
import numpy as np
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
import torch

import copy
cpu_count = 4
use_gpu = False
@dataclass
class LGBOpt:
    num_threads: any = hp.choice('num_threads', [cpu_count])
    num_leaves: any = hp.choice('num_leaves', [64])
    metric: any = hp.choice('metric', ['binary_error'])
    num_round: any = hp.choice('num_rounds', [1000])
    objective: any = hp.choice('objective', ['binary'])
    learning_rate: any = hp.uniform('learning_rate', 0.01, 0.1)
    feature_fraction: any = hp.uniform('feature_fraction', 0.5, 1.0)
    bagging_fraction: any = hp.uniform('bagging_fraction', 0.8, 1.0)
    device_type: any = hp.choice('device_tpye', ['gpu']) if use_gpu else hp.choice('device_type',
                                                                                   ['cpu'])
    boosting: any = hp.choice('boosting', ['gbdt', 'dart', 'goss'])
    extra_trees: any = hp.choice('extra_tress', [False, True])
    drop_rate: any = hp.uniform('drop_rate', 0, 0.2)
    uniform_drop: any = hp.choice('uniform_drop', [True, False])
    lambda_l1: any = hp.uniform('lambda_l1', 0, 10)  # TODO: Check range
    lambda_l2: any = hp.uniform('lambda_l2', 0, 10)  # TODO: Check range
    min_gain_to_split: any = hp.uniform('min_gain_to_split', 0, 1)  # TODO: Check range
    min_data_in_bin = hp.choice('min_data_in_bin', [3, 5, 10, 15, 20, 50])

    @staticmethod
    def get_common_params():
        return {'num_thread': 4, 'num_leaves': 12, 'metric': 'binary', 'objective': 'binary',
                'num_round': 1000, 'learning_rate': 0.01, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
    

In [10]:
class FitterBase(object):
    def __init__(self, label, metric, max_eval=100, opt=None):
        self.label = label
        self.metric = metric
        self.opt_params = dict()
        self.max_eval = max_eval
        self.opt = opt

    def get_loss(self, y, y_pred):
        if self.metric == 'error':
            return 1 - accuracy_score(y, y_pred)
        elif self.metric == 'precision':
            return 1 - precision_score(y, y_pred)
        elif self.metric == 'recall':
            return 1 - recall_score(y, y_pred)
        elif self.metric == 'macro_f1':
            return 1 - f1_score(y, y_pred, average='macro')
        elif self.metric == 'micro_f1':
            return 1 - f1_score(y, y_pred, average='micro')
        elif self.metric == 'auc':  # TODO: Add a warning checking if y_predict is all [0, 1], it should be probability
            return 1 - roc_auc_score(y, y_pred)
        else:
            raise Exception("Not implemented yet.")


In [11]:
class LGBFitter(FitterBase):
    def __init__(self, label='label', metric='error', opt: LGBOpt = None, max_eval=100):
        super(LGBFitter, self).__init__(label, metric, max_eval)
        if opt is not None:
            self.opt = opt
        else:
            self.opt = LGBOpt()
        self.best_round = None
        self.clf = None

    def train(self, train_df, eval_df, params=None, use_best_eval=True):
        self.best_round = None
        dtrain = lgb.Dataset(train_df.drop(columns=[self.label]), train_df[self.label])
        deval = lgb.Dataset(eval_df.drop(columns=[self.label]), eval_df[self.label])
        evallist = [dtrain, deval]
        if params is None:
            use_params = deepcopy(self.opt_params)
        else:
            use_params = deepcopy(params)

        num_round = use_params.pop('num_round')
        if use_best_eval:
            with io.StringIO() as buf, redirect_stdout(buf):
                self.clf = lgb.train(use_params, dtrain, num_round, valid_sets=evallist)
                output = buf.getvalue().split("\n")
            min_error = np.inf
            min_index = 0
            for idx in range(len(output) - 1):
                if len(output[idx].split("\t")) == 3:
                    temp = float(output[idx].split("\t")[2].split(":")[1])
                    if min_error > temp:
                        min_error = temp
                        min_index = int(output[idx].split("\t")[0][1:-1])
            print("The minimum is attained in round %d" % (min_index + 1))
            self.best_round = min_index + 1
            return output
        else:
            with io.StringIO() as buf, redirect_stdout(buf):
                self.clf = lgb.train(use_params, dtrain, num_round, valid_sets=evallist)
                output = buf.getvalue().split("\n")
            self.best_round = num_round
            return output

    def search(self, train_df, eval_df, use_best_eval=True):
        self.opt_params = dict()

        def train_impl(params):
            self.train(train_df, eval_df, params, use_best_eval)
            if self.metric == 'auc':
                y_pred = self.clf.predict(eval_df.drop(columns=[self.label]), num_iteration=self.best_round)
            else:
                y_pred = (self.clf.predict(eval_df.drop(columns=[self.label]),
                                           num_iteration=self.best_round) > 0.5).astype(int)
            return self.get_loss(eval_df[self.label], y_pred)

        self.opt_params = fmin(train_impl, asdict(self.opt), algo=tpe.suggest, max_evals=self.max_eval)

    def search_k_fold(self, k_fold, data, use_best_eval=True):
        self.opt_params = dict()

        def train_impl_nfold(params):
            loss = list()
            for train_id, eval_id in k_fold.split(data):
                train_df = data.loc[train_id]
                eval_df = data.loc[eval_id]
                self.train(train_df, eval_df, params, use_best_eval)
                if self.metric == 'auc':
                    y_pred = self.clf.predict(eval_df.drop(columns=[self.label]), num_iteration=self.best_round)
                else:
                    y_pred = (self.clf.predict(eval_df.drop(columns=[self.label]),
                                               num_iteration=self.best_round) > 0.5).astype(int)
                loss.append(self.get_loss(eval_df[self.label], y_pred))
            return np.mean(loss)

        self.opt_params = fmin(train_impl_nfold, asdict(self.opt), algo=tpe.suggest, max_evals=self.max_eval)

    def train_k_fold(self, k_fold, train_data, test_data, params=None, drop_test_y=True, use_best_eval=True):
        acc_result = list()
        train_pred = np.empty(train_data.shape[0])
        test_pred = np.empty(test_data.shape[0])
        if drop_test_y:
            dtest = test_data.drop(columns=self.label)
        else:
            dtest = test_data

        models = list()
        for train_id, eval_id in k_fold.split(train_data):
            train_df = train_data.loc[train_id]
            eval_df = train_data.loc[eval_id]
            self.train(train_df, eval_df, params, use_best_eval)
            models.append(copy.deepcopy(self.clf))
            train_pred[eval_id] = self.clf.predict(eval_df.drop(columns=self.label), num_iteration=self.best_round)
            if self.metric == 'auc':
                y_pred = self.clf.predict(eval_df.drop(columns=[self.label]), num_iteration=self.best_round)
            else:
                y_pred = (self.clf.predict(eval_df.drop(columns=[self.label]),
                                           num_iteration=self.best_round) > 0.5).astype(int)
            acc_result.append(self.get_loss(eval_df[self.label], y_pred))
            test_pred += self.clf.predict(dtest, num_iteration=self.best_round)
        test_pred /= k_fold.n_splits
        return train_pred, test_pred, acc_result, models

In [12]:
fitter = LGBFitter(label='loan_status')

In [13]:
params = {'num_thread': 4, 'num_leaves': 4, 'metric': 'binary', 'objective': 'binary',
                'num_round': 2000, 'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}

In [14]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5)

In [15]:
train_pred,test_pred,acc,model = fitter.train_k_fold(kfold, tran_shaffle, test, params = params)

The minimum is attained in round 974
The minimum is attained in round 1042
The minimum is attained in round 1782
The minimum is attained in round 1302
The minimum is attained in round 773


In [ ]:
import numpy as np
print(params)
print(np.mean(acc))

{'num_thread': 4, 'num_leaves': 4, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.07944


In [ ]:
1- accuracy_score(test['loan_status'], (test_pred > 0.5).astype(np.int64))

0.08279999999999998

In [ ]:
from sklearn.model_selection import KFold
import numpy as np
for num_leaves in [2,4,6,8]:
  params = {'num_thread': 4, 'num_leaves': num_leaves, 'metric': 'binary', 'objective': 'binary',
                'num_round': 4000, 'learning_rate': 0.01, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
  kfold = KFold(n_splits=5)
  _,_,acc,_ = fitter.train_k_fold(kfold, train, test, params = params)
  print(params)
  print(np.mean(acc))

In [ ]:
from sklearn.model_selection import KFold
import numpy as np
for learning_rate in [0.045, 0.04, 0.035, 0.03, 0.025]:
  params = {'num_thread': 4, 'num_leaves': 4, 'metric': 'binary', 'objective': 'binary',
                'num_round': 4000, 'learning_rate': learning_rate, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
  kfold = KFold(n_splits=5)
  _,_,acc,_ = fitter.train_k_fold(kfold, train, test, params = params)
  print(params)
  print(np.mean(acc))

The minimum is attained in round 459
The minimum is attained in round 634
The minimum is attained in round 801
The minimum is attained in round 597
The minimum is attained in round 324
{'num_thread': 4, 'num_leaves': 4, 'metric': 'binary', 'objective': 'binary', 'num_round': 4000, 'learning_rate': 0.045, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.07986000000000001
The minimum is attained in round 534
The minimum is attained in round 772
The minimum is attained in round 499
The minimum is attained in round 489
The minimum is attained in round 365
{'num_thread': 4, 'num_leaves': 4, 'metric': 'binary', 'objective': 'binary', 'num_round': 4000, 'learning_rate': 0.04, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.07996
The minimum is attained in round 664
The minimum is attained in round 582
The minimum is attained in round 1023
The minimum is attained in round 816
The minimum is attained in round 480
{'num_thread': 4, 'num_leaves': 4, 'metric': 'binary', 'objective': 'binary

第一次调参：

{'num_thread': 4, 'num_leaves': 60, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.08198000000000001**

The minimum is attained in round 348
The minimum is attained in round 242
The minimum is attained in round 279
The minimum is attained in round 234
The minimum is attained in round 234

第二次调参：因为第一次在第348轮就收敛了，总共2000轮，所以我决定这一步降低学习率
{'num_thread': 4, 'num_leaves': 60, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': **0.01**, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.0819**

The minimum is attained in round 709
The minimum is attained in round 486
The minimum is attained in round 558
The minimum is attained in round 459
The minimum is attained in round 498

第三次调参：因为第二次调参降低了学习率，错误率没有什么变化，因此这次考虑降低叶子结点数
{'num_thread': 4, 'num_leaves': **12**, 'metric': 'binary', 'objective': 'binary', 'num_round': 4000, 'learning_rate': 0.01, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.08008**

第四次调参：因为上次调参，减少叶子结点数，错误率有所下降，所以这次考虑通过for循环设置不同的叶子节点数
The minimum is attained in round 960
The minimum is attained in round 1113
The minimum is attained in round 903
The minimum is attained in round 1444
The minimum is attained in round 741
{'num_thread': 4, 'num_leaves': **12**, 'metric': 'binary', 'objective': 'binary', 'num_round': 4000, 'learning_rate': 0.01, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.08008**

The minimum is attained in round 733
The minimum is attained in round 565
The minimum is attained in round 581
The minimum is attained in round 613
The minimum is attained in round 539
{'num_thread': 4, 'num_leaves': **32**, 'metric': 'binary', 'objective': 'binary', 'num_round': 4000, 'learning_rate': 0.01, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.08102000000000001**

The minimum is attained in round 748
The minimum is attained in round 489
The minimum is attained in round 552
The minimum is attained in round 539
The minimum is attained in round 465
{'num_thread': 4, 'num_leaves': **64**, 'metric': 'binary', 'objective': 'binary', 'num_round': 4000, 'learning_rate': 0.01, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.08179999999999998**

The minimum is attained in round 681
The minimum is attained in round 499
The minimum is attained in round 498
The minimum is attained in round 454
The minimum is attained in round 465
{'num_thread': 4, 'num_leaves': **80**, 'metric': 'binary', 'objective': 'binary', 'num_round': 4000, 'learning_rate': 0.01, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.08185999999999997**

The minimum is attained in round 641
The minimum is attained in round 472
The minimum is attained in round 474
The minimum is attained in round 440
The minimum is attained in round 443
{'num_thread': 4, 'num_leaves': **128**, 'metric': 'binary', 'objective': 'binary', 'num_round': 4000, 'learning_rate': 0.01, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.08282**

The minimum is attained in round 500
The minimum is attained in round 398
The minimum is attained in round 425
The minimum is attained in round 381
The minimum is attained in round 379
{'num_thread': 4, 'num_leaves': **256**, 'metric': 'binary', 'objective': 'binary', 'num_round': 4000, 'learning_rate': 0.01, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.08278**


第五次调参：上次调参发现叶子结点数也高，模型错误率越高，所以这次考虑通过for循环尝试低一点的叶子结点数
The minimum is attained in round 3996
The minimum is attained in round 3993
The minimum is attained in round 3982
The minimum is attained in round 4001
The minimum is attained in round 3979
{'num_thread': 4, 'num_leaves': **2**, 'metric': 'binary', 'objective': 'binary', 'num_round': 4000, 'learning_rate': 0.01, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.08008**

The minimum is attained in round 1848
The minimum is attained in round 2100
The minimum is attained in round 3616
The minimum is attained in round 2075
The minimum is attained in round 1539
{'num_thread': 4, 'num_leaves': **4**, 'metric': 'binary', 'objective': 'binary', 'num_round': 4000, 'learning_rate': 0.01, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.07973999999999999**

The minimum is attained in round 1374
The minimum is attained in round 1514
The minimum is attained in round 1341
The minimum is attained in round 1605
The minimum is attained in round 1038
{'num_thread': 4, 'num_leaves': **6**, 'metric': 'binary', 'objective': 'binary', 'num_round': 4000, 'learning_rate': 0.01, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.08013999999999999**

The minimum is attained in round 1039
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:99: RuntimeWarning: invalid value encountered in add
The minimum is attained in round 1258
The minimum is attained in round 907
The minimum is attained in round 1379
The minimum is attained in round 767
{'num_thread': 4, 'num_leaves': **8**, 'metric': 'binary', 'objective': 'binary', 'num_round': 4000, 'learning_rate': 0.01, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.08002000000000001**

第六次调参：选取上次错误率最低的叶子节点数，并减小学习率
The minimum is attained in round 3830
The minimum is attained in round 3900
The minimum is attained in round 3988
The minimum is attained in round 3894
The minimum is attained in round 3194
{'num_thread': 4, 'num_leaves': **4**, 'metric': 'binary', 'objective': 'binary', 'num_round': 4000, 'learning_rate': 0.005, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.07968**

第七次调参：发现上次收敛的轮数比较高，所以尝试升高num_round
{'num_thread': 4, 'num_leaves': 4, 'metric': 'binary', 'objective': 'binary', 'num_round': **6000**, 'learning_rate': 0.005, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.07982000000000003**


第八次调参：尝试稍微提高一点学习率，看看会有什么变化
The minimum is attained in round 2292
The minimum is attained in round 2511
The minimum is attained in round 4243
The minimum is attained in round 3457
The minimum is attained in round 1897
{'num_thread': 4, 'num_leaves': 4, 'metric': 'binary', 'objective': 'binary', 'num_round': 6000, 'learning_rate': **0.008**, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.07949999999999997**

第九次调参：上一步发现错误率降低，于是继续提高学习率
The minimum is attained in round 1848
The minimum is attained in round 2100
The minimum is attained in round 3616
The minimum is attained in round 2075
The minimum is attained in round 1539

{'num_thread': 4, 'num_leaves': 4, 'metric': 'binary', 'objective': 'binary', 'num_round': 6000, 'learning_rate': **0.01**, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.07973999999999999**

第十次调参：上一步发现提高学习率，错误率升高了，于是选取错误率低的学习率0.008的周围0.006，尝试一下
The minimum is attained in round 3475
The minimum is attained in round 3351
The minimum is attained in round 5884
The minimum is attained in round 4302
The minimum is attained in round 2651
{'num_thread': 4, 'num_leaves': 4, 'metric': 'binary', 'objective': 'binary', 'num_round': 6000, 'learning_rate': **0.006**, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.07971999999999999**

第十一次调参：上一步错误率上升了，于是选择学习率0.007尝试一下
The minimum is attained in round 3019
The minimum is attained in round 2847
The minimum is attained in round 4911
The minimum is attained in round 3861
The minimum is attained in round 2124
{'num_thread': 4, 'num_leaves': 4, 'metric': 'binary', 'objective': 'binary', 'num_round': 6000, 'learning_rate': **0.007**, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.0797**

第十二次调参：上几步调整学习率发现，除了0.008的错误率低外，周围其他值都比它高，于是我想调出这个圈子，尝试一个稍大一点的学习率0.02
The minimum is attained in round 974
The minimum is attained in round 1042
The minimum is attained in round 1782
The minimum is attained in round 1302
The minimum is attained in round 773
 {'num_thread': 4, 'num_leaves': 4, 'metric': 'binary', 'objective': 'binary', 'num_round': 5000, 'learning_rate': **0.02**, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.07944**

第十三次调参：上一步发现学习率升高到0.02，错误率有所下降，但收敛速度相对num_round有点快，于是尝试减小num_round
The minimum is attained in round 974
The minimum is attained in round 1042
The minimum is attained in round 1782
The minimum is attained in round 1302
The minimum is attained in round 773
{'num_thread': 4, 'num_leaves': 4, 'metric': 'binary', 'objective': 'binary', 'num_round': 3000, 'learning_rate': **0.02**, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.07944**

第十四次调参：上一步调整num_round发现结果也没有什么变化，于是尝试降低学习试试
The minimum is attained in round 1331
The minimum is attained in round 1395
The minimum is attained in round 2629
The minimum is attained in round 1557
The minimum is attained in round 1119
{'num_thread': 4, 'num_leaves': 4, 'metric': 'binary', 'objective': 'binary', 'num_round': 3000, 'learning_rate': **0.015**, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.07971999999999999**

第十五次调参：上一步学习率降到0.015，错误率上升，这次尝试0.018
The minimum is attained in round 1131
The minimum is attained in round 1176
The minimum is attained in round 2058
The minimum is attained in round 1516
The minimum is attained in round 841
{'num_thread': 4, 'num_leaves': 4, 'metric': 'binary', 'objective': 'binary', 'num_round': 4000, 'learning_rate': **0.018**, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.07974**

 第十六次调参：前两步学习率叫目前最佳学习率0.02稍低，错误率都有所上升，这次尝试多个较高学习率
The minimum is attained in round 373
The minimum is attained in round 433
The minimum is attained in round 461
The minimum is attained in round 606
The minimum is attained in round 332
{'num_thread': 4, 'num_leaves': 4, 'metric': 'binary', 'objective': 'binary', 'num_round': 4000, 'learning_rate': **0.05**, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.07990000000000001**

The minimum is attained in round 459
The minimum is attained in round 634
The minimum is attained in round 801
The minimum is attained in round 597
The minimum is attained in round 324
{'num_thread': 4, 'num_leaves': 4, 'metric': 'binary', 'objective': 'binary', 'num_round': 4000, 'learning_rate': **0.045**, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.07986000000000001**

The minimum is attained in round 534
The minimum is attained in round 772
The minimum is attained in round 499
The minimum is attained in round 489
The minimum is attained in round 365
{'num_thread': 4, 'num_leaves': 4, 'metric': 'binary', 'objective': 'binary', 'num_round': 4000, 'learning_rate': **0.04**, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.07996**

The minimum is attained in round 664
The minimum is attained in round 582
The minimum is attained in round 1023
The minimum is attained in round 816
The minimum is attained in round 480
{'num_thread': 4, 'num_leaves': 4, 'metric': 'binary', 'objective': 'binary', 'num_round': 4000, 'learning_rate': **0.035**, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.07982**

The minimum is attained in round 658
The minimum is attained in round 660
The minimum is attained in round 1143
The minimum is attained in round 683
The minimum is attained in round 515
{'num_thread': 4, 'num_leaves': 4, 'metric': 'binary', 'objective': 'binary', 'num_round': 4000, 'learning_rate': **0.03**, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.0799**

The minimum is attained in round 823
The minimum is attained in round 787
The minimum is attained in round 1469
The minimum is attained in round 1305
The minimum is attained in round 600
{'num_thread': 4, 'num_leaves': 4, 'metric': 'binary', 'objective': 'binary', 'num_round': 4000, 'learning_rate': **0.025**, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.08**

第十七次调参：目前收敛还是有点快，降低num_round
The minimum is attained in round 974
The minimum is attained in round 1042
The minimum is attained in round 1782
The minimum is attained in round 1302
The minimum is attained in round 773
{'num_thread': 4, 'num_leaves': 4, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.07944**

# 总结：
目前最优的参数为：
{'num_thread': 4, 'num_leaves': 4, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.07944**